In [116]:
import random

In [4]:
def modexp(a, p, m):
  if p == 0:
    return 1
  if p == 1:
    return a % m
  a_p = a
  i = 1
  while i*2 <= p:
    a_p = (a_p * a_p) % m
    i *= 2
  return (a_p * modexp(a, p-i, m)) % m

### Challenge 33

In [84]:
p = 37
g = 5

a = random.randint(0, p-1)
A = modexp(g, a, p)

b = random.randint(0, p-1)
B = modexp(g, b, p)

s = modexp(B, a, p)
s_ = modexp(A, b, p)

assert s == s_
print(s)

36


In [85]:
p = int(
"""
0xffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024
e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd
3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec
6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f
24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361
c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552
bb9ed529077096966d670c354e4abc9804f1746c08ca237327fff
fffffffffffff
""".replace("\n", ""), 0)

g = 2

a = random.randint(0, p-1)
A = modexp(g, a, p)

b = random.randint(0, p-1)
B = modexp(g, b, p)

s = modexp(B, a, p)
s_ = modexp(A, b, p)

assert s == s_
print(s)

2261578341463150962949917447080753488146348936583348253351029011333326328132331760639833595923143550573868258030909252914197695502309971595062194036655611024972877785222316969852040485520080871952561540243810419781665595916390219356501623822391431801618072024787212065278435172564363911941220427743256796216564616080648149640035019362171312645437404506362598867087505429112075442623077634688502591963459639700676024392872015590242569865903298088855229889546359038


### Challenge 34

In [6]:
from Crypto.Cipher import AES
from Crypto.Hash import SHA

def sha1(msg):
    h = SHA.new()
    h.update(msg)
    return h.digest()

def random_bytes(len):
    return bytes([random.randint(0,255) for _ in range(len)])

In [102]:
class A_:
    
    def __init__(self):
        self.p = p
        self.g = g
        self.a = random.randint(0, self.p-1)
        self.A = modexp(self.g, self.a, self.p)
        self.iv = random_bytes(16)
        self.msg = b"attack at dawn.."

    def send_params(self):
        return (self.p, self.g, self.A)
    
    def receive_params(self, B):
        self.B = B
        self.s = modexp(self.B, self.a, self.p)
        self.k = sha1(self.s.to_bytes(256, 'big'))[:16]
    
    def send_msg(self):
        cipher = AES.new(self.k, AES.MODE_CBC, self.iv)
        return cipher.encrypt(self.msg) + self.iv
    
    def receive_msg(self, cmsg):
        cipher = AES.new(self.k, AES.MODE_CBC, self.iv)
        assert cipher.decrypt(cmsg[:-16]) == self.msg

        
class B_:
        
    def receive_params(self, params):
        self.p, self.g, self.A = params
        self.b = random.randint(0, self.p-1)
        self.B = modexp(self.g, self.b, self.p)
        self.s = modexp(self.A, self.b, self.p)
        self.k = sha1(self.s.to_bytes(256, 'big'))[:16]
        
    def send_params(self):
        return self.B
    
    def receive_msg(self, cmsg):
        self.iv = cmsg[-16:]
        cipher = AES.new(self.k, AES.MODE_CBC, self.iv)
        self.msg = cipher.decrypt(cmsg[:-16])
    
    def send_msg(self):
        cipher = AES.new(self.k, AES.MODE_CBC, self.iv)
        return cipher.encrypt(self.msg) + self.iv

        
class M_:
    
    def receive_params_A(self, params):
        self.p, self.g, self.A = params

    def send_params_B(self, p=None, g=None, A=None):
        if p is None: p = self.p
        if g is None: g = self.g
        if A is None: A = self.A
        return (p, g, A)
        
    def receive_params_B(self, B):
        self.B = B
    
    def send_params_A(self, B=None):
        if B is None: B = self.B
        return B
    
    def receive_msg_A(self, cmsg):
        self.cmsg = cmsg
        
    def send_msg_B(self):
        return self.cmsg
    
    def receive_msg_B(self, cmsg):
        self.cmsg = cmsg
        
    def send_msg_A(self):
        return self.cmsg

In [103]:
A = A_()
B = B_()

# A->B
B.receive_params(A.send_params())
# B->A
A.receive_params(B.send_params())
# A->B
B.receive_msg(A.send_msg())
# B->A
A.receive_msg(B.send_msg())

In [104]:
A = A_()
B = B_()
M = M_()

# A->M
M.receive_params_A(A.send_params())
# M->B
B.receive_params(M.send_params_B(A=M.p))
# B->M
M.receive_params_B(B.send_params())
# M->A
A.receive_params(M.send_params_A(B=M.p))
# A->M
M.receive_msg_A(A.send_msg())
# M->B
B.receive_msg(M.send_msg_B())
# B->M
M.receive_msg_B(B.send_msg())
# M->A
A.receive_msg(A.send_msg())

cipher = AES.new(sha1((0).to_bytes(256, 'big'))[:16], AES.MODE_CBC, M.cmsg[-16:])
print(cipher.decrypt(M.cmsg[:-16]))

b'attack at dawn..'


### Challenge 35

In [113]:
A = A_()
B = B_()
M = M_()

def mitm_attack(s, p_icpt=None, g_icpt=None, A_icpt=None, B_icpt=None):
    # A->M
    M.receive_params_A(A.send_params())
    # M->B
    B.receive_params(M.send_params_B(p=p_icpt, g=g_icpt, A=A_icpt))
    # B->M
    M.receive_params_B(B.send_params())
    # M->A
    A.receive_params(M.send_params_A(B=B_icpt))

    # A->M
    M.receive_msg_A(A.send_msg())
    # M->B
    B.receive_msg(M.send_msg_B())
    # B->M
    M.receive_msg_B(B.send_msg())
    # M->A
    A.receive_msg(A.send_msg())

    cipher = AES.new(sha1(s.to_bytes(256, 'big'))[:16], AES.MODE_CBC, M.cmsg[-16:])
    print(cipher.decrypt(M.cmsg[:-16]))   

In [114]:
mitm_attack(s=1, g_icpt=1)

b'attack at dawn..'


In [115]:
mitm_attack(s=0, g_icpt=p)

b'attack at dawn..'


For the case when $g=p-1$, note that (for example) $g^a = (p-1)^a = \sum_{i=0}^{a} (-1)^a p^a \equiv (-1)^a \pmod p$ and so $s\equiv\pm1 \pmod p$

In [116]:
mitm_attack(s=1, g_icpt=p-1)
mitm_attack(s=p-1, g_icpt=p-1)

b'attack at dawn..'
b'\xd4\xbeo\x8ah\xb3\x0b.\x80\xa5\xe2[\x83S\x08\x95'


### Challenge 36

In [95]:
from Crypto.Hash import SHA256, HMAC

def sha256(msg):
    h = SHA256.new()
    h.update(msg)
    return h.digest()

def hmac_sha256(key, msg):
    h = HMAC.new(key, digestmod=SHA256)
    h.update(msg)
    return h.digest()

In [96]:
def agree_init(obj):
    obj.N = p
    obj.g = 2
    obj.k = 3
    obj.I = b'me@there.com'
    obj.P = b'challenge36'


class S_:
    
    def __init__(self):
        agree_init(self)
        self.salt = random_bytes(16)
        xH = sha256(self.salt + self.P)
        x = int.from_bytes(xH, byteorder='big')
        self.v = modexp(g, x, self.N)
        
    def receive_params(self, params):
        I, self.A = params
        assert I == self.I
        
    def send_params(self):
        self.b = random.randint(0, self.N-1)
        self.B = self.k*self.v + modexp(self.g, self.b, self.N)
        return (self.salt, self.B)
        
    def receive_mac(self, mac):
        uH = sha256(self.A.to_bytes(256, 'big') + self.B.to_bytes(256, 'big'))
        u = int.from_bytes(uH, byteorder='big')
        self.S = modexp(self.A * modexp(self.v, u, self.N), self.b, self.N)
        self.K = sha256(self.S.to_bytes(256, 'big'))
        
        self.match = mac == hmac_sha256(self.K, self.salt)
            
    def send_confirm(self):
        return "Auth OK" if self.match else "Auth Fail"

    
class C_:
    
    def __init__(self):
        agree_init(self)
        
    def send_params(self, my_A=None):
        self.a = random.randint(0, self.N-1)
        self.A = modexp(g, self.a, self.N)
        return (self.I, self.A if my_A is None else my_A)
    
    def receive_params(self, params):
        self.salt, self.B = params
        
    def send_mac(self, my_mac=None):
        if my_mac is not None:
            return my_mac
        uH = sha256(self.A.to_bytes(256, 'big') + self.B.to_bytes(256, 'big'))
        u = int.from_bytes(uH, byteorder='big')
        xH = sha256(self.salt + self.P)
        x = int.from_bytes(xH, byteorder='big')
        self.S = modexp((self.B - self.k * modexp(self.g, x, self.N)), self.a + u*x, self.N)
        self.K = sha256(self.S.to_bytes(256, 'big'))
        
        return hmac_sha256(self.K, self.salt)
    
    def receive_confirm(self, msg):
        print(msg)

In [97]:
S = S_()
C = C_()

# C->S
S.receive_params(C.send_params())
# S->C
C.receive_params(S.send_params())
# C->S
S.receive_mac(C.send_mac())
# S->C
C.receive_confirm(S.send_confirm())

Auth OK


### Challenge 37

In [98]:
S = S_()
C = C_()

for my_A in [0, C.N, 2*C.N]:
    # C->S
    S.receive_params(C.send_params(my_A))
    # S->C
    C.receive_params(S.send_params())
    # C->S
    my_K = sha256((0).to_bytes(256, 'big'))
    S.receive_mac(C.send_mac(my_mac=hmac_sha256(my_K, C.salt)))
    # S->C
    C.receive_confirm(S.send_confirm())

Auth OK
Auth OK
Auth OK


### Challenge 38

I will not implement the client-server model here because the idea is simple. The attacker generates his own key $b$ and sends to C the original salt and $u$ plus $B=g^b$ to C.

In [28]:
N = p
# S->M and M->C
salt = random_bytes(16)
u = int.from_bytes(random_bytes(16), 'big')
# M->C
b = random.randint(0, N-1)
B = modexp(g, b, N)

Then he receives the MAC from C...

In [47]:
# C's private computations
password = b'Aaron'
xH = sha256(salt + password)
x = int.from_bytes(xH, byteorder='big')
S = modexp(B, a + u*x, N)
K = sha256(S.to_bytes(256, 'big'))

# C->M
mac = hmac_sha256(K, salt)

...and performs an (awfully slow and wishful) dictionary attack:

In [50]:
for word in open('/usr/share/dict/words'):
    xH = sha256(salt + bytes(word.rstrip(), 'ascii'))
    x = int.from_bytes(xH, byteorder='big')
    S = modexp(B, a + u*x, N)
    K = sha256(S.to_bytes(256, 'big'))
    if hmac_sha256(K, salt) == mac:
        print(word.rstrip())
        break

Aaron


### Challenge 39

In [98]:
from Crypto.Util.number import getPrime

In [298]:
def egcd(m, n):
    if m == 0:
        return (n, 0, 1)
    else:
        g, x, y = egcd(n % m, m)
        return (g, y - (n//m) * x, x)
    
def invmod(x, n):
    g, x, _ = egcd(x, n)
    if g == 1:
        return x % n
    else: raise Exception('{} and {} are not rel. prime'.format(x, n))

In [316]:
class PrivateStore:
    d = None

def init_rsa(e=None):
    while True:
        p = getPrime(512)
        q = getPrime(512)
        n = p*q
        et = (p-1)*(q-1)
        if e is None:
            while e is None or egcd(e, et)[0] != 1:
                e = random.randint(0, n-1)
        try:
            PrivateStore.d = invmod(e, et)
        except:
            continue
        return (e, n)

e, n = init_rsa(e=3);

In [317]:
def rsa_encrypt(m, e, n):
    return modexp(m, e, n)

def rsa_decrypt(c, d, n):
    return modexp(c, d, n)

In [318]:
assert rsa_decrypt(rsa_encrypt(42, e, n), PrivateStore.d, n) == 42

In [319]:
msg = "attack at dawn!" * 8  # make sure message is long enough; otherwise (padless) encryption does nothing!
msg = int.from_bytes(msg.encode('utf-8'), 'big')
rsa_decrypt(rsa_encrypt(msg, e, n), PrivateStore.d, n).to_bytes(128, 'big')

b'\x00\x00\x00\x00\x00\x00\x00\x00attack at dawn!attack at dawn!attack at dawn!attack at dawn!attack at dawn!attack at dawn!attack at dawn!attack at dawn!'

### Challenge 40

In [320]:
def cube_root(x):
    lo = 0
    hi = x
    while True:
        if lo**3 == x: return lo
        if hi**3 == x: return hi
        if ((lo + hi) // 2) ** 3 < x:
            lo = (lo + hi) // 2
        else:
            hi = (lo + hi) // 2

In [321]:
n = [0]*3
c = [0]*3
for i in range(3):
    _, n[i] = init_rsa(e=3)
    c[i] = rsa_encrypt(msg, 3, n[i])

In [322]:
ms = [0]*3
ms[0] = n[1] * n[2]
ms[1] = n[2] * n[0]
ms[2] = n[0] * n[1]
N = n[0] * n[1] * n[2]

res = sum([c[i] * ms[i] * invmod(ms[i], n[i]) for i in range(3)]) % N
cube_root(res).to_bytes(128, 'big')

b'\x00\x00\x00\x00\x00\x00\x00\x00attack at dawn!attack at dawn!attack at dawn!attack at dawn!attack at dawn!attack at dawn!attack at dawn!attack at dawn!'